In [1]:
from get_data import get_data


get_data()

Unzipping data


## Preparing dataset:

In [2]:
import numpy as np
import pandas as pd


DATASET_PATH = "data/UCI HAR Dataset/"
TRAIN = "train/"
TEST = "test/"
INPUT_TYPES = [
    "body_acc_x_",
    "body_acc_y_",
    "body_acc_z_",
    "body_gyro_x_",
    "body_gyro_y_",
    "body_gyro_z_",
    "total_acc_x_",
    "total_acc_y_",
    "total_acc_z_"
]

X_train = np.dstack([
    pd.read_csv(DATASET_PATH + TRAIN + "Inertial Signals/" + signal + "train.txt", header=None, delim_whitespace=True).values for signal in INPUT_TYPES
])
X_test = np.dstack([
    pd.read_csv(DATASET_PATH + TEST + "Inertial Signals/" + signal + "test.txt", header=None, delim_whitespace=True).values for signal in INPUT_TYPES
])

y_raw_train = pd.read_csv(DATASET_PATH + TRAIN + "y_train.txt", header=None, delim_whitespace=True).values
y_raw_test = pd.read_csv(DATASET_PATH + TEST + "y_test.txt", header=None, delim_whitespace=True).values


In [3]:
from sklearn.preprocessing import OneHotEncoder


encoder = OneHotEncoder()

y_train = encoder.fit_transform(y_raw_train).toarray()
y_test = encoder.transform(y_raw_test).toarray()

In [4]:
import numpy as np


# Some debugging info
print("Some useful info to get an insight on dataset's shape and normalisation:")
print("(X shape, y shape, every X's mean, every X's standard deviation)")
print(X_train.shape, y_train.shape, np.mean(X_train), np.std(X_train))
print(X_test.shape, y_test.shape, np.mean(X_test), np.std(X_test))
print("The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.")

Some useful info to get an insight on dataset's shape and normalisation:
(X shape, y shape, every X's mean, every X's standard deviation)
(7352, 128, 9) (7352, 6) 0.10206605723804434 0.4021651763827929
(2947, 128, 9) (2947, 6) 0.09913989069610848 0.39567084061541474
The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.


In [5]:
from tensorflow.keras import models, layers
from tensorflow.keras.regularizers import l2


l2_value = 0.0015

lstm = models.Sequential([
    layers.Dense(32, activation='relu', kernel_regularizer=l2(l2_value), bias_regularizer=l2(l2_value)),
    layers.LSTM(32, input_shape=(128, 9), return_sequences=True, kernel_regularizer=l2(l2_value), bias_regularizer=l2(l2_value), recurrent_regularizer=l2(l2_value)),
    layers.LSTM(32, kernel_regularizer=l2(l2_value), bias_regularizer=l2(l2_value), recurrent_regularizer=l2(l2_value)),
    layers.Dense(6, activation="softmax", kernel_regularizer=l2(l2_value), bias_regularizer=l2(l2_value))
])

lstm.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
history = lstm.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test))

2021-08-26 17:37:58.681630: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-26 17:37:58.688533: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7fca65f20d70 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-08-26 17:37:58.688548: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): Host, Default Version
2021-08-26 17:37:58.784258: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/30
230/230 [==============================] - 20s 74ms/step - loss: 1.2802 - accuracy: 0.5760 - val_loss: 1.0598 - val_accuracy: 0.6474
Epoch 2/30
230/230 [==============================] - 17s 73ms/step - loss: 0.8585 - accuracy: 0.7257 - val_loss: 0.9233 - val_accuracy: 0.6973
Epoch 3/30
230/230 [==============================] - 17s 76ms/step - loss: 0.8508 - accuracy: 0.7247 - val_loss: 1.0297 - val_accuracy: 0.6614
Epoch 4/30
230/230 [==============================] - 17s 75ms/step - loss: 0.8906 - accuracy: 0.7137 - val_loss: 0.9640 - val_accuracy: 0.6787
Epoch 5/30
230/230 [==============================] - 17s 74ms/step - loss: 0.7745 - accuracy: 0.7644 - val_loss: 0.9495 - val_accuracy: 0.6960
Epoch 6/30
230/230 [==============================] - 17s 73ms/step - loss: 0.6594 - accuracy: 0.8192 - val_loss: 0.8314 - val_accuracy: 0.7679
Epoch 7/30
230/230 [==============================] - 17s 75ms/step - loss: 0.5634 - accuracy: 0.8779 - val_loss: 0.8404 - val_accuracy: